In [1]:
import numpy as np
import pandas as pd

from scipy import stats

import datetime
from pandas.tseries.frequencies import to_offset

import niftyutils
from niftyutils import load_nifty_data

import matplotlib.pyplot as plt
import mplfinance as fplt

In [2]:
start_date = datetime.datetime(2005,8,1)
end_date = datetime.datetime(2020,8,29)
nifty_data = load_nifty_data(start_date,end_date)

In [3]:
weekly_nifty_data = nifty_data.resample('W').agg(niftyutils.OHLC_CONVERSION_DICT)
weekly_nifty_data.index = weekly_nifty_data.index - to_offset('6D')
weekly_nifty_data.tail(3)

,Open,High,Low,Close,Volume
Date,,,,,
2020-08-10,11270.25,11373.60,11111.45,11178.4,2.895408e+09
2020-08-17,11248.90,11460.35,11144.50,11371.6,3.299731e+09
2020-08-24,11412.00,11686.05,11410.65,11647.6,3.441270e+09


In [4]:
weekly_returns = (weekly_nifty_data['Close']/weekly_nifty_data['Close'].shift(1) - 1)*100
weekly_returns = weekly_returns.dropna()

In [6]:
two_weeks_history_data = pd.concat([weekly_returns.shift(1).rename('Previous Week'), weekly_returns.rename('Current Week')], axis=1)
two_weeks_history_data = two_weeks_history_data.dropna()

In [8]:
reversal_data = two_weeks_history_data[(two_weeks_history_data['Current Week'] > 3.0) &
                                       (two_weeks_history_data['Previous Week'] < -3.5)].copy()

In [9]:
reversal_data

,Previous Week,Current Week
Date,,
2005-10-31,-5.225575,4.447227
2006-07-24,-5.710215,6.308998
2007-10-22,-3.922995,9.337910
2007-11-12,-4.536950,4.301417
2007-12-24,-4.649702,5.431371
2008-02-11,-3.703042,3.565186
2008-03-24,-3.621097,8.046656
2008-05-12,-4.697601,3.514230
2008-10-27,-15.949713,11.671827


In [11]:
pct_change_after_bear_week = []
for ix, pct_chage in reversal_data.iterrows():
    pct_change_by_week = []
    for i in range(1,5):
        after_ix = weekly_returns.index.get_loc(ix) + i
        ret = weekly_returns[after_ix]
        pct_change_by_week.append(ret)
    
    pct_change_after_bear_week.append(pct_change_by_week)

In [13]:
bear_week_study_data = pd.DataFrame(pct_change_after_bear_week, columns=['1st','2nd','3rd','4th'],
                                                  index = reversal_data.index)

In [16]:
bear_week_study_data

,1st,2nd,3rd,4th
Date,,,,
2005-10-31,5.357475,2.801483,2.419801,0.540349
2006-07-24,1.467676,3.072322,2.516530,0.869889
2007-10-22,4.035214,-4.536950,4.301417,-5.049223
2007-11-12,-5.049223,2.748458,3.670990,1.228596
2007-12-24,3.200816,-1.182602,-7.980516,-5.642999
2008-02-11,-3.623489,2.206134,-8.651287,-0.540699
2008-03-24,-5.969243,2.814719,3.779982,3.091723
2008-05-12,-4.093879,-1.545522,-4.975257,-2.392065
2008-10-27,3.028833,-5.470905,-4.159624,2.288886


In [21]:
result_data = pd.concat([reversal_data,bear_week_study_data], axis=1)
result_data

,Previous Week,Current Week,1st,2nd,3rd,4th
Date,,,,,,
2005-10-31,-5.225575,4.447227,5.357475,2.801483,2.419801,0.540349
2006-07-24,-5.710215,6.308998,1.467676,3.072322,2.516530,0.869889
2007-10-22,-3.922995,9.337910,4.035214,-4.536950,4.301417,-5.049223
2007-11-12,-4.536950,4.301417,-5.049223,2.748458,3.670990,1.228596
2007-12-24,-4.649702,5.431371,3.200816,-1.182602,-7.980516,-5.642999
2008-02-11,-3.703042,3.565186,-3.623489,2.206134,-8.651287,-0.540699
2008-03-24,-3.621097,8.046656,-5.969243,2.814719,3.779982,3.091723
2008-05-12,-4.697601,3.514230,-4.093879,-1.545522,-4.975257,-2.392065
2008-10-27,-15.949713,11.671827,3.028833,-5.470905,-4.159624,2.288886


In [26]:
result_data[(abs(result_data['Previous Week']) > result_data['Current Week']) &
             (result_data['Previous Week'] > -7.0)]

,Previous Week,Current Week,1st,2nd,3rd,4th
Date,,,,,,
2005-10-31,-5.225575,4.447227,5.357475,2.801483,2.419801,0.540349
2007-11-12,-4.536950,4.301417,-5.049223,2.748458,3.670990,1.228596
2008-02-11,-3.703042,3.565186,-3.623489,2.206134,-8.651287,-0.540699
2008-05-12,-4.697601,3.514230,-4.093879,-1.545522,-4.975257,-2.392065
2009-03-09,-5.192409,3.782226,3.228831,10.744376,3.294034,4.079662
2016-02-15,-6.785194,3.291816,-2.510141,6.481027,0.331982,1.253628
